## Additional reading

Two textbooks which have explanations for many of the algorithms we will be discussing:

* [Cormen, Leiserson, Rivest, Stein, "Introduction to algorithms"](https://drive.google.com/file/d/1xVe7q3PNclptq7QnOTL3AT2YWOe9F3sj/view?usp=drive_link)
* [Kleinberg, Tardos, "Algorithm Design"](https://drive.google.com/file/d/1wKy-wGuCYfRx1DH48vZHl5JA4nkhOmwC/view?usp=share_link)

Introduction to Turing machines and complexity theory:

* [Sipser, "Introduction to the theory of computation"](https://drive.google.com/file/d/1VBmwnz_5IPJyLuMgF2OpahSD_Izk3u5U/view?usp=drive_link)

A gentle introduction to the ideas of computability and complexity:

* [Harel, Feldman, "Algorithmics: Spirit of Computing"](https://drive.google.com/file/d/1m9-9Kw8EM4sahuWgF8UONigmyXFad6Mz/view?usp=share_link)

## Exercise: Finding a largest subproduct

Let ``A`` be a nonempty list containing a sequence of $n$ real numbers $a_0,\ldots,a_{n-1}$.
For $0\le i\le j\le n-1$, let the *$(i,j)$-subproduct* be
$P(i,j):=\prod_{k=i}^j a_k$.
Write a Python function with input ``A`` and output which is a triple $(i,j,P(i,j))$ for $i,j$ which achieve the maximum
possible value of $P(i,j)$.

We will test the code on the following inputs:

In [15]:
import numpy as np
np.random.seed(3)
INPUTS = [
    [0.1, 0.3, 2, 0.3, 1.1, 2],
    [-0.1, 0.3, 2, -0.3, -1.1, 2],
    [-2],
    [1, 5],
    list( np.random.normal(size=100) ),
    list( np.random.normal(size=1000) ),
    list( np.random.normal(size=1000000) ),
]

In [16]:
def subproduct(B, i, j):       # Time complexity: 
    res = 1                    # O(1) time.
    for k in range(i, j+1):    # j-i+1 iterations of the loop
        #res = res * B[k]
        res *= B[k]            # Every iteration of the loop takes O(1) time.
    return res                 # O(1) time.

# Total time complexity: O(1) + (j-i+1)*O(1) + O(1) = O(j-i+1)

In [17]:
INPUTS[0]

[0.1, 0.3, 2, 0.3, 1.1, 2]

In [18]:
subproduct([2, 3, 4, 5, 6], 3, 3)

5

In [21]:
def largest_subproduct_n3(A):        # Total complexity: O(n^3)
    res = subproduct(A, 0, 0)        # O(1)
    res_i = 0
    res_j = 0
    n = len(A)                       # O(1) [ Python functionality ]
    
    for i in range(n):             # Outer loop executed n times.              
        for j in range(i, n):      # For a given i, the inner loop is executed n-i times.
            prod = subproduct(A, i, j)    # O(j-i+1)
            if prod > res:                # O(1)
                res = prod
                res_i = i
                res_j = j
    return res_i, res_j, res       # O(1)

The total running time of `largest_subproduct_n3` is therefore  $O(1) + O\left(\sum_{i=0}^{n-1} \sum_{j=i}^{n-1} j-i+1\right)$.

Let $A(n) := \sum_{i=0}^{n-1}\sum_{j=i}^{n-1} j-i+1$. We will now show that $A(n)\in O(n^3)$ and $n^3\in O(A(n))$. Therefore, the time complexity of `smallest_subproduct_n3` has the order of magnitude $n^3$. In other words, the time complexity is $O(n^3)$ and even $\Theta(n^3)$.

Indeed, $A(n) = \sum_{i=0}^{n-1}\sum_{j=i}^{n-1} j-i+1 \le \sum_{i=0}^{n-1}\sum_{j=i}^{n-1} n \le n^3$, hence $A(n)\in O(n^3)$.

On the other hand,

$$A(n) \ge \sum_{0\le i\le n/4}\sum_{j=i}^{n-1} j-i+1 \ge \sum_{0\le i\le n/4}\sum_{n/2\le j\le n-1} j-i+1\ge \sum_{0\le i\le n/4}\sum_{n/2\le j\le n-1}n/4 
\ge \frac{n}{4}\cdot \left(\frac{n}{2}-1\right)\cdot\frac{n}{4} = n^3/32-n^2/16.$$

But there exists $n_0$ such that for $n\ge n_0$ it holds
$n^3/32-n^2/16\ge n^3/64$, which implies $n^3\in O(A(n))$.

In [23]:
%%time
for A in INPUTS[:-1]:
    print( largest_subproduct_n3(A) )

(4, 5, 2.2)
(2, 2, 2)
(0, 0, -2)
(0, 1, 5)
(52, 61, np.float64(5.240889037066749))
(577, 582, np.float64(9.000207769761909))
CPU times: user 8.85 s, sys: 33.4 ms, total: 8.88 s
Wall time: 8.9 s


**$O(n^2)$ solution**

We can use the fact that for $i<j\le n-1$ it holds $P(i,j)=P(i,j-1)\cdot a_j$ to speed up the computation of $P(i,j)$:

In [24]:
def largest_subproduct_n2(A):     # Total complexity: O(n^2)
    res = subproduct(A, 0, 0)     # O(1)
    res_i = 0
    res_j = 0
    n = len(A)                    # O(1) [ Python functionality ]
    
    for i in range(n):                           
        prod = 1
        for j in range(i, n):      
            prod = prod*A[j]
            if prod > res:                
                res = prod
                res_i = i
                res_j = j
    return res_i, res_j, res       # O(1)

The time complexity of `smallest_subproduct_n2` can be estimated as

$\sum_{i=0}^{n-1}\sum_{j=i}^{n-1} O(1)$

$ = O(\binom{n}{2}+n) = O(n^2).$

Note that the number of all pairs $(i,j)$ such that $0\le i\le j\le n-1$ is $\binom{n}{2}+n$.

This is because $|\{0\le i<j\le n-1: i<j\}|=\binom{n}{2}$ and there are $n$ more pairs with $i=j$.

In [26]:
%%time
for A in INPUTS[:-1]:
    print( largest_subproduct_n2(A) )

(4, 5, 2.2)
(2, 2, 2)
(0, 0, -2)
(0, 1, 5)
(52, 61, np.float64(5.240889037066749))
(577, 582, np.float64(9.000207769761909))
CPU times: user 59.2 ms, sys: 2.72 ms, total: 62 ms
Wall time: 61 ms


**$O(n)$ solution**

As will be presented in the lecture, we can implement a more clever $O(n)$ algorithm. This is an example of a method called __dynamic programming.__

However, the algorithm below is correct only for arrays such that $a_i\ge 0$ for every $i$.

In [28]:
def largest_subproduct_n(A):
    n = len(A)                          # O(1)
    Q = [A[0]]                          # O(1)
    for j in range(1, n):               # n-1 times
        new_Q = max(A[j], A[j]*Q[j-1])     # O(1)
        Q.append(new_Q)                    # O(1)
    
    result = Q[0]                       # O(1)
    for j in range(1, n):               # O(n)
        result = max(result, Q[j])         # O(1) in each iteration
    return result                       # O(1)
#   return min(Q)                       # That would also take O(n).

In [29]:
import numpy as np
np.random.seed(3)
INPUTS = [
    [0.1, 0.3, 2, 0.3, 1.1, 2],
    np.abs([-0.1, 0.3, 2, -0.3, -1.1, 2]),
    [2],
    [1, 5],
    list( np.abs(np.random.normal(size=100)) ),
    list( np.abs(np.random.normal(size=1000)) ),
    list( np.abs(np.random.normal(size=1000000)) ),
]

In [30]:
%%time
for A in INPUTS:
    print( largest_subproduct_n(A) )

2.2
2.2
2
5
5.240889037066749
9.12027615840467
6152.68646331551
CPU times: user 194 ms, sys: 7.32 ms, total: 201 ms
Wall time: 201 ms


**Questions/exercises about the $O(n)$ algorithm**:
* Can you improve the algorithm so that it is correct for $a_i\in\mathbb{R}$, not just $a_i\ge 0$?
* Can you improve it to return $(i,j,P(i,j))$ and not just $P(i,j)$?
* Do you need to store all of the $Q$ array? Can we reduce the memory complexity to $O(1)$?